<a href="https://colab.research.google.com/github/Thaslim/MLDemo/blob/master/tfliteconvert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly

     |████████████████████████████████| 392.2MB 40kB/s 
     |████████████████████████████████| 471kB 45.5MB/s 
     |████████████████████████████████| 10.6MB 35.6MB/s 


In [2]:

import pandas as pd
#import splitfolders
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers, optimizers
print(tf.__version__)
print("Eagerly enabled: ", tf.executing_eagerly())

INFO:tensorflow:Using local port 18806
INFO:tensorflow:Using local port 15078
INFO:tensorflow:Using local port 22328
INFO:tensorflow:Using local port 22317
INFO:tensorflow:Using local port 17588
INFO:tensorflow:Using local port 21030
INFO:tensorflow:Using local port 22894
INFO:tensorflow:Using local port 19522
INFO:tensorflow:Using local port 22324
INFO:tensorflow:Using local port 19469
2.4.0-dev20201012
Eagerly enabled:  True


In [3]:
#define LSTM model with skip connection
def LSTM_net():
    N_CLASSES=4

    i = layers.Input(shape=(79, 20), name='input')
    x = layers.Masking()(i)
    x = layers.LayerNormalization(name='layer_norm')(x)
    s = layers.TimeDistributed(layers.Dense(64, activation='tanh'),
                        name='td_dense_tanh')(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True),
                             name='bidirectional_lstm')(s)
    x = layers.concatenate([s, x], axis=2, name='skip_connection')
    x = layers.Dense(64, activation='relu', name='dense_1_relu')(x)
    x = layers.MaxPooling1D(name='max_pool_1d')(x)
    x = layers.Dense(32, activation='relu', name='dense_2_relu')(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(rate=0.5, name='dropout')(x)
    x = layers.Dense(32, activation='relu',
                         activity_regularizer=regularizers.l2(0.001),
                         name='dense_3_relu')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)

    model = Model(inputs=i, outputs=o, name='long_short_term_memory')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [4]:
LSTM_net().summary()

Model: "long_short_term_memory"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 79, 20)]     0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 79, 20)       0           input[0][0]                      
__________________________________________________________________________________________________
layer_norm (LayerNormalization) (None, 79, 20)       40          masking[0][0]                    
__________________________________________________________________________________________________
td_dense_tanh (TimeDistributed) (None, 79, 64)       1344        layer_norm[0][0]                 
_____________________________________________________________________________

In [5]:
model = LSTM_net()

In [6]:
#### The Model is an already trained MobileNet from tf.keras.applications ##########
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=True
converter.experimental_new_converter =True
tflite_model = converter.convert()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2310: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:1388: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


INFO:tensorflow:Assets written to: /tmp/tmpphetjlpu/assets


In [7]:
open("converted_model.tflite", "wb").write(tflite_model)
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

In [8]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
print(input_shape)


[ 1 79 20]


In [9]:
import numpy as np

In [10]:
input_data = np.array( np.random.random((79,20)), dtype=np.float32)

In [11]:
input_data = np.expand_dims(input_data, axis=0)
input_data.shape

(1, 79, 20)

In [12]:
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

RuntimeError: ignored